In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from tqdm.notebook import tqdm

In [2]:
# upper bound/lower bound
db1 = np.ones(9) * 200
ub1 = np.zeros(9)
db2 = np.ones(9) * 200
ub2 = np.zeros(9)
env = AngryBird()
for j in range(1000):
    state = env.reset()
    for i in range(11):
        if i < 9:
            state, cost, done, _ = env.step(np.zeros(2))
            x1 = state[1]
            x2 = state[2]
            db1[i] = min(db1[i], x1)
            ub1[i] = max(ub1[i], x1)
            db2[i] = min(db2[i], x2)
            ub2[i] = max(ub2[i], x2)
        else:
            state, cost, done, _ = env.step(np.zeros(2))
print(ub1 - db1)
print(ub2 - db2)

[ 2.66724279  7.3457522  13.72595516 22.12843189 32.12757262 44.91872352
 60.16468421 75.53584017 93.58662238]
[ 2.64527711  7.4361341  15.69990845 22.58114271 31.68210988 41.71424926
 54.36703461 70.8923421  91.51282736]


In [3]:
# Polynome de deg 3
def init_mat():
    fmat = []
    fmat.append(np.zeros((1, 1, 2)))
    for i in range(1, 10):
        fmat.append(np.zeros((i, i, 8)))
    return fmat

def force_from_coeff(x1, x2, coeffs):
    x1 /= 100
    x2 /= 50
    a0, a1, a2, a3, b0, b1, b2, b3 = coeffs
    force1 = a0 + a1*x1 + a2*x1**2 + a3*x1**3
    force2 = b0 + b1*x2 + b2*x2**2 + b3*x2**3
    return np.array([force1, force2])

def force_from_mat(mat, state):
    t, x1, x2 = state
    t = int(t)
    if t == 0:
        return mat[0][0][0]
    else:
        i1 = min(max(int((x1 - db1[t-1]) / (ub1[t-1]-db1[t-1]) * t), 0), t-1)
        i2 = min(max(int((x2 - db2[t-1]) / (ub2[t-1]-db2[t-1]) * t), 0), t-1)
        coeffs = mat[t][i1][i2]
        #print(i1, i2)
        return force_from_coeff(x1, x2, coeffs)

# Force de rappel
def main(state):
    t = state[0]
    return -state[1:]+np.array([10*t, 20*t-2*t**2])

In [4]:
umat = init_mat()
vmat = init_mat()

In [5]:
def rand_place():
    i = np.random.randint(0, 2282)
    #print(i)
    if i < 2:
        return 0, 0, 0, i
    else:
        j = 2
        for a in range(1, 10):
            for b in range(a):
                for c in range(a):
                    for d in range(8):
                        if i == j:
                            return a, b, c, d
                        else:
                            j += 1

def rand_places(i):
    places = []
    for _ in range(i):
        places.append(rand_place())
    return places

def tups_update(mat, places, lr, j):
    try:
        for tup in places:
            a, b, c, d = tup
            mat[a][b][c][d] += lr * j
    except:
        print(places)

In [6]:
%matplotlib notebook
env = AngryBird()

lr = 0.01
step = 0
m = 100
rewards = np.zeros(m)
average_reward = 300.
all_rewards = []
while average_reward > 80.:
    if (step+1)%100==0:
        print("Step {}, last cost = {}, mean over 100 rounds = {}".format(step+1, average_reward, np.mean(all_rewards[-100:])))
    places = rand_places(1)
    tups_update(vmat, places, lr, 1)
    j = 0.
    for z in range(m):
        state = env.reset()
        local_reward = 0.
        for i in range(10):
            u_force = force_from_mat(umat, state) + main(state)
            v_force = force_from_mat(vmat, state)
            state, reward, done, _ = env.step(u_force, v_force)
            local_reward += reward
        j += env.j_prime_MC()
        _, reward, done, _ = env.step(np.zeros(2))
        local_reward += reward
        rewards[z] = local_reward
    j /= m
    j = max(-5, min(5, j))
    tups_update(umat, places, lr, -j)
    tups_update(vmat, places, lr, -1)
    step+=1
    average_reward = np.mean(rewards)
    all_rewards.append(average_reward)
    np.save("rewards", all_rewards)

Step 100, last cost = 161.48517456457978, mean over 100 rounds = 142.26276753123767
Step 200, last cost = 157.53441135582585, mean over 100 rounds = 146.96590002356152
Step 300, last cost = 139.2478680008518, mean over 100 rounds = 145.95944146278526
Step 400, last cost = 129.91448116713786, mean over 100 rounds = 143.8980837223454
Step 500, last cost = 117.14300579437221, mean over 100 rounds = 142.27342906424494
Step 600, last cost = 137.17699398833113, mean over 100 rounds = 141.29403925291612
Step 700, last cost = 185.42535821004196, mean over 100 rounds = 144.61643458765712
Step 800, last cost = 134.9548921016102, mean over 100 rounds = 145.55924154870806
Step 900, last cost = 143.54660283814255, mean over 100 rounds = 143.68718672886365
Step 1000, last cost = 130.0355852838374, mean over 100 rounds = 147.37493793681458
Step 1100, last cost = 137.76044378477945, mean over 100 rounds = 144.8669623370082
Step 1200, last cost = 131.55921417206736, mean over 100 rounds = 145.666282492

KeyboardInterrupt: 

#### Save
db1, ub1, db2, ub2, umat